UK MONETARY POLICY 2019-2025

## Objectives

* The objectives of this project are to:

Collect official UK macroeconomic data from reliable public sources

Apply Python-based ETL (Extract, Transform, Load) techniques

Clean and preprocess time-series data for analysis

Convert datasets to a common quarterly frequency for alignment

Analyse trends in inflation, Official Bank Rate and GDP between 2019 and 2025
## Inputs

* The following data inputs required: 
1. Bank of England Policy Interest Rate : time series data on the UK nominal policy interest rate 
2. Consumer Price Index(CPI): UK CPI, used as a measure inflation
3. Gross Domestic Product(GDP) : UK GDP data, used to represent economic activity. GDP

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [8]:
import os
current_dir = os.getcwd() #
current_dir

'\\\\talktalk\\redirectedfolders\\F.Afolabi\\Documents\\VSCode1\\UK_MonetaryPolicy_2019'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [73]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [74]:
current_dir = os.getcwd()
current_dir

'\\\\talktalk\\redirectedfolders'

# Section 1
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

Section 1 Extraction: Load the Dataset

In [4]:
import pandas as pd

cpi = pd.read_csv(r"\\talktalk\redirectedfolders\F.Afolabi\Documents\VSCode1\inflation.csv")
print(cpi.head())

# Display basic information about the dataset, I would take the variable one by one; manipulating CPI first; 
cpi.info() 

print(cpi.shape)  # Looking at the shape of CPI

   Date  CPI_Inflation
0  1996           68.8
1  1997           70.1
2  1998           71.2
3  1999           72.1
4  2000           72.7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1280 non-null   object 
 1   CPI_Inflation  634 non-null    float64
dtypes: float64(1), object(1)
memory usage: 20.1+ KB
(1280, 2)


In [ ]:
# Generate a summary of the statistics
print(cpi.describe())

       CPI_Inflation
count     634.000000
mean       86.401577
std        22.090846
min        48.400000
25%        70.500000
50%        81.150000
75%       100.550000
max       139.800000


In [12]:
# Checking for missing values and data type
print(cpi.isnull().sum())
print(cpi.dtypes)

Date             0
CPI_Inflation    0
Year             0
dtype: int64
Date             datetime64[ns]
CPI_Inflation           float64
Year                      int32
dtype: object


In [18]:
# Converting Date to datetime
cpi['Date'] = pd.to_datetime(cpi['Date'], errors='coerce')

In [23]:
# Filtering the data between 2019 and 2025 for CPI for the analysis and the scope of this project
cpi = cpi[(cpi['Date'] >= '2019-01-01') & (cpi['Date'] <= '2025-12-31')]
print(cpi.head())
cpi.info()

         Date  CPI_Inflation  Year
23 2019-01-01          107.8  2019
24 2020-01-01          108.7  2020
25 2021-01-01          111.6  2021
26 2022-01-01          121.7  2022
27 2023-01-01          130.5  2023
<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 23 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           7 non-null      datetime64[ns]
 1   CPI_Inflation  7 non-null      float64       
 2   Year           7 non-null      int32         
dtypes: datetime64[ns](1), float64(1), int32(1)
memory usage: 196.0 bytes


In [27]:
#filling missing values using interpolation method
cpi['CPI_Inflation'] = cpi['CPI_Inflation'].interpolate()


Converting CPI From Annual to Quarterly Frequency

In [30]:
#Annual aggregation of CPI Inflation
cpi['Year'] = cpi['Date'].dt.year
cpi_annual = (cpi.groupby('Year', as_index=False).agg({'CPI_Inflation': 'mean'}))
print(cpi_annual)

   Year  CPI_Inflation
0  2019          107.8
1  2020          108.7
2  2021          111.6
3  2022          121.7
4  2023          130.5
5  2024          133.9
6  2025          133.9


In [36]:
#Converting annual CPI to quarterly CPI by forward filling the values

q_index = pd.period_range('2019Q1', '2025Q4', freq='Q')

cpi_q = cpi_annual.copy()

# Creating a  Date column from Year
cpi_q['Date'] = pd.PeriodIndex(cpi_q['Year'].astype(str) + 'Q1', freq='Q')

cpi_q = cpi_q[['Date', 'CPI_Inflation']] # Selecting only the necessary columns



#Reindexing and forward filling
cpi_q = (cpi_q.set_index('Date').reindex(q_index).ffill().reset_index().rename(columns={'index': 'Date'}))









In [70]:
print(len(cpi_q))
print(cpi_q.head())
print(cpi_q.tail())

28
     Date  CPI_Inflation
0  2019Q1          107.8
1  2019Q2          107.8
2  2019Q3          107.8
3  2019Q4          107.8
4  2020Q1          108.7
      Date  CPI_Inflation
23  2024Q4          133.9
24  2025Q1          133.9
25  2025Q2          133.9
26  2025Q3          133.9
27  2025Q4          133.9


In [103]:
cpi_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype        
---  ------         --------------  -----        
 0   Date           28 non-null     period[Q-DEC]
 1   CPI_Inflation  28 non-null     float64      
dtypes: float64(1), period[Q-DEC](1)
memory usage: 580.0 bytes


In [135]:
 cpi_q_cleaned = cpi_q.to_csv('cpi_q_cleaned.csv', index=False) # Saving the cleaned CPI data to a new CSV file, for further analysis

Loading the Official Bank Rate to be cleaned. 

In [166]:
boe_rate = pd.read_csv('../Dataset/Raw/Bank_rate_boe.csv')
boe_rate.head()
    

,Date,Official Bank Rate [a] [b] IUDBEDR
0,15-Dec-25,4.0
1,12-Dec-25,4.0
2,11-Dec-25,4.0
3,10-Dec-25,4.0
4,09-Dec-25,4.0


In [196]:
boe_rate.info() #Looking at the data types and missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       124 non-null    datetime64[ns]
 1   Bank_Rate  124 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.1 KB


In [197]:
# Rename the Colum, I rename the column to be consistent across the dataset

boe_rate.columns = ['Date', 'Bank_Rate']

In [225]:
boe_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       124 non-null    datetime64[ns]
 1   Bank_Rate  124 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.1 KB


In [252]:
#Converting the Date to datetime also to quarterly frequency, this will be consistent with CPI data and GDP data
boe_rate['Date'] = pd.to_datetime(boe_rate['Date'])
boe_rate.set_index('Date', inplace=True)
boe_rate = boe_rate.resample('QS').mean()
print(boe_rate.head())

            Bank_Rate
Date                 
1995-01-01   6.453125
1995-04-01   6.625000
1995-07-01   6.625000
1995-10-01   6.581349
1996-01-01   6.125000


In [278]:
# Filtering the data between 2019 and 2025 for Bank Rate  for the analysis and the scope of this project, date reset to allow filtering
boe_rate.reset_index(inplace=True)
boe_rate_filtered = boe_rate[(boe_rate['Date'] >= '2019-01-01') & (boe_rate['Date'] <= '2025-12-31')]
print(boe_rate_filtered.head())
boe_rate_filtered.info()

     index       Date  Bank_Rate
96      96 2019-01-01   0.750000
97      97 2019-04-01   0.750000
98      98 2019-07-01   0.750000
99      99 2019-10-01   0.750000
100    100 2020-01-01   0.611719
<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 96 to 123
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   index      28 non-null     int64         
 1   Date       28 non-null     datetime64[ns]
 2   Bank_Rate  28 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 896.0 bytes


In [304]:
# Setting  Bank Rate to Quarterly frequency.
boe_rate_filtered = boe_rate_filtered.resample('QS').mean()
print(boe_rate_filtered.head())


            index  Bank_Rate
Date                        
2019-01-01   96.0   0.750000
2019-04-01   97.0   0.750000
2019-07-01   98.0   0.750000
2019-10-01   99.0   0.750000
2020-01-01  100.0   0.611719


In [305]:
boe_rate_filtered.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28 entries, 2019-01-01 to 2025-10-01
Freq: QS-JAN
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      28 non-null     float64
 1   Bank_Rate  28 non-null     float64
dtypes: float64(2)
memory usage: 672.0 bytes


In [306]:
# Checking for missing values
print(boe_rate_filtered.isnull().sum())

index        0
Bank_Rate    0
dtype: int64


In [307]:
#Checking for missing Quaterly periods
boe_rate_filtered = boe_rate_filtered.asfreq('QS')
print(boe_rate_filtered.isnull().sum())


index        0
Bank_Rate    0
dtype: int64


In [308]:
#print Bank Rtae info
boe_rate_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28 entries, 2019-01-01 to 2025-10-01
Freq: QS-JAN
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      28 non-null     float64
 1   Bank_Rate  28 non-null     float64
dtypes: float64(2)
memory usage: 672.0 bytes


In [327]:
# Resetting index
boe_rate_filtered=boe_rate_filtered.reset_index() 

In [347]:
#Dropping the index column because it is not needed
boe_rate_filtered = boe_rate_filtered.drop(columns=['index'])

In [385]:
# Convert Date to Period for consistency
boe_rate_filtered['Date'] = boe_rate_filtered['Date'].dt.to_period('Q')


In [386]:
# saved the cleaned Bank Rate data to a new CSV file, for further analysis
boe_rate_cleaned = boe_rate_filtered.to_csv('boe_rate_cleaned.csv', index=True)

In [387]:
print(boe_rate_filtered.head())

     Date  Bank_Rate
0  2019Q1   0.750000
1  2019Q2   0.750000
2  2019Q3   0.750000
3  2019Q4   0.750000
4  2020Q1   0.611719


In [575]:
# Loading GDP data
gdp = pd.read_csv('../Dataset/Raw/GDP.csv')
gdp.head(5)


,Date,"PP: Household final consumption expenditure, growth Q on Q","PP: NPISH final consumption expenditure, growth Q on Q","PP: General Government final consumption expenditure, growth Q on Q","PP: Gross Capital Formation, growth Q on Q","PP: Gross fixed capital formation, growth Q on Q","PP: Business investment, growth Q on Q","PP: Exports, growth Q on Q","PP: Less imports, growth Q on Q","PP: Household final consumption expenditure, growth Q on Q yr ago",...,"Deflator: Gross fixed capital formation, growth Q on Q yr ago","Deflator: Exports, growth Q on Q yr ago","Deflator: Less imports, growth Q on Q yr ago","Deflator: Household final consumption expenditure, growth annual","Deflator: NPISH final consumption expenditure, growth annual","Deflator: General Government final consumption expenditure, growth annual","Deflator: Gross Capital Formation, growth annual","Deflator: Gross fixed capital formation, growth annual","Deflator: Exports, growth annual","Deflator: Less imports, growth annual"
0,CDID,A24M,A24X,A24Y,A25I,A25J,A25M,A264,A268,A26C,...,ZZ8Y,ZZ8Z,ZZ92,ZZ93,ZZ94,ZZ95,ZZ96,ZZ97,ZZ98,ZZ99
1,PreUnit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Release Date,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,...,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025,30/09/2025
4,Next release,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,...,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25,13-Nov-25


In [597]:
print(gdp.info)

<bound method DataFrame.info of              Date PP: Household final consumption expenditure, growth Q on Q  \
0            CDID                                               A24M           
1         PreUnit                                                NaN           
2            Unit                                                NaN           
3    Release Date                                         30/09/2025           
4    Next release                                          13-Nov-25           
..            ...                                                ...           
398       2024 Q2                                               -0.1           
399       2024 Q3                                                0.5           
400       2024 Q4                                                0.1           
401       2025 Q1                                                0.4           
402       2025 Q2                                                0.1           

    PP:

In [618]:
print(gdp.columns)

Index(['Date', 'PP: Household final consumption expenditure, growth Q on Q',
       'PP: NPISH final consumption expenditure, growth Q on Q',
       'PP: General Government final consumption expenditure, growth Q on Q',
       'PP: Gross Capital Formation, growth Q on Q',
       'PP: Gross fixed capital formation, growth Q on Q',
       'PP: Business investment, growth Q on Q', 'PP: Exports, growth Q on Q',
       'PP: Less imports, growth Q on Q',
       'PP: Household final consumption expenditure, growth Q on Q yr ago',
       ...
       'Deflator: Gross fixed capital formation, growth Q on Q yr ago',
       'Deflator: Exports, growth Q on Q yr ago',
       'Deflator: Less imports, growth Q on Q yr ago',
       'Deflator: Household final consumption expenditure, growth annual',
       'Deflator: NPISH final consumption expenditure, growth annual',
       'Deflator: General Government final consumption expenditure, growth annual',
       'Deflator: Gross Capital Formation, growth ann

In [638]:
#Finding all columns containing 'Gross' or 'GDP' becasue they are relevant to our analysis 
gdp_columns = [col for col in gdp.columns if 'Gross' in col or 'GDP' in col]
print(gdp_columns)

['PP: Gross Capital Formation, growth Q on Q', 'PP: Gross fixed capital formation, growth Q on Q', 'PP: Gross Capital Formation, growth Q on Q yr ago', 'PP: Gross fixed capital formation, growth Q on Q yr ago', 'PP: Gross Capital Formation, growth annual', 'PP: Gross fixed capital formation, growth annual', 'Gross Domestic Product at market prices, q-on-q1 growth: CP: NSA m', 'Gross Domestic Product at market prices, q-on-q4 growth: CP: NSA m', 'Gross Domestic Product', 'Gross Value Added (Average) at basic prices: CP SA m', 'Gross Value Added at basic prices:chained volume measures:Seasonally adjusted m', 'Gross National Income: Current price: Seasonally adjusted m', 'Gross Domestic Product at market prices: CP: NSA m', 'Gross Domestic Product (Expenditure) current basic prices: aligned:SA m', 'Gross Domestic Product (Expenditure) chained volume measure SA m', 'Gross Domestic Product (Income) current basic prices: aligned: SA m', 'Gross Value Added at basic prices: Implied deflator:SA

In [657]:
for col in gdp.columns:
    print(repr(col))

'Date'
'PP: Household final consumption expenditure, growth Q on Q'
'PP: NPISH final consumption expenditure, growth Q on Q'
'PP: General Government final consumption expenditure, growth Q on Q'
'PP: Gross Capital Formation, growth Q on Q'
'PP: Gross fixed capital formation, growth Q on Q'
'PP: Business investment, growth Q on Q'
'PP: Exports, growth Q on Q'
'PP: Less imports, growth Q on Q'
'PP: Household final consumption expenditure, growth Q on Q yr ago'
'PP: NPISH final consumption expenditure, growth Q on Q yr ago'
'PP: General Government final consumption expenditure, growth Q on Q yr ago'
'PP: Gross Capital Formation, growth Q on Q yr ago'
'PP: Gross fixed capital formation, growth Q on Q yr ago'
'PP: Business investment, growth Q on Q yr ago'
'PP: Exports, growth Q on Q yr ago'
'PP: Less imports, growth Q on Q yr ago'
'PP: Household final consumption expenditure, growth annual'
'PP: NPISH final consumption expenditure, growth annual'
'PP: General Government final consumption e

In [675]:
gdp.columns = (
    gdp.columns
      .astype(str)
      .str.encode('latin1', errors='ignore').str.decode('utf-8', errors='ignore')
      .str.replace(r'\s+', ' ', regex=True)
      .str.replace('\\', '', regex=False)
      .str.strip()
)

In [692]:
print(list(gdp.columns))


['Date', 'PP: Household final consumption expenditure, growth Q on Q', 'PP: NPISH final consumption expenditure, growth Q on Q', 'PP: General Government final consumption expenditure, growth Q on Q', 'PP: Gross Capital Formation, growth Q on Q', 'PP: Gross fixed capital formation, growth Q on Q', 'PP: Business investment, growth Q on Q', 'PP: Exports, growth Q on Q', 'PP: Less imports, growth Q on Q', 'PP: Household final consumption expenditure, growth Q on Q yr ago', 'PP: NPISH final consumption expenditure, growth Q on Q yr ago', 'PP: General Government final consumption expenditure, growth Q on Q yr ago', 'PP: Gross Capital Formation, growth Q on Q yr ago', 'PP: Gross fixed capital formation, growth Q on Q yr ago', 'PP: Business investment, growth Q on Q yr ago', 'PP: Exports, growth Q on Q yr ago', 'PP: Less imports, growth Q on Q yr ago', 'PP: Household final consumption expenditure, growth annual', 'PP: NPISH final consumption expenditure, growth annual', 'PP: General Government

In [693]:
for i, col in enumerate(gdp.columns):
    print(i, repr(col))

0 'Date'
1 'PP: Household final consumption expenditure, growth Q on Q'
2 'PP: NPISH final consumption expenditure, growth Q on Q'
3 'PP: General Government final consumption expenditure, growth Q on Q'
4 'PP: Gross Capital Formation, growth Q on Q'
5 'PP: Gross fixed capital formation, growth Q on Q'
6 'PP: Business investment, growth Q on Q'
7 'PP: Exports, growth Q on Q'
8 'PP: Less imports, growth Q on Q'
9 'PP: Household final consumption expenditure, growth Q on Q yr ago'
10 'PP: NPISH final consumption expenditure, growth Q on Q yr ago'
11 'PP: General Government final consumption expenditure, growth Q on Q yr ago'
12 'PP: Gross Capital Formation, growth Q on Q yr ago'
13 'PP: Gross fixed capital formation, growth Q on Q yr ago'
14 'PP: Business investment, growth Q on Q yr ago'
15 'PP: Exports, growth Q on Q yr ago'
16 'PP: Less imports, growth Q on Q yr ago'
17 'PP: Household final consumption expenditure, growth annual'
18 'PP: NPISH final consumption expenditure, growth annu

In [708]:
# SLICING: I need to create a new dataframe with only the relevant columns

gdp_clean = gdp[['Date', 'Gross Domestic Product']].copy() 

# Rename columns for clarity
gdp_clean = gdp_clean.rename(columns={'Gross Domestic Product': 'GDP_Level'})

In [709]:
gdp_clean.head()
gdp_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       403 non-null    object
 1   GDP_Level  364 non-null    object
dtypes: object(2)
memory usage: 6.4+ KB


In [724]:
#Converting 'Date' to string to enable filtering



#converting GDP to numeric
gdp_clean['GDP_Level'] = pd.to_numeric(gdp_clean['GDP_Level'], errors='coerce')






In [736]:
# filtering for the period 2019 Q1 to 2025 Q4
gdp_clean = gdp_clean[(gdp_clean['Date'] >= '2019Q1') & (gdp_clean['Date'] <= '2025Q4')].copy()


In [747]:
gdp_clean.info()
gdp_clean.head()
gdp_clean.tail()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 377 to 402
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype        
---  ------     --------------  -----        
 0   Date       26 non-null     period[Q-DEC]
 1   GDP_Level  26 non-null     float64      
dtypes: float64(1), period[Q-DEC](1)
memory usage: 624.0 bytes


,Date,GDP_Level
398,2024Q2,695793.0
399,2024Q3,697153.0
400,2024Q4,698832.0
401,2025Q1,703458.0
402,2025Q2,705403.0


In [757]:
print(gdp_clean['Date'].min(), gdp_clean['Date'].max())
print(gdp_clean['Date'].dtype)

2019Q1 2025Q2
period[Q-DEC]


GDP dates were converted to quarterly period format for alignment with both Bank rate and Inflation(CPI)

In [777]:
gdp_clean_cleaned = gdp_clean.to_csv('gdp_cleaned.csv', index=False) # Saving the cleaned GDP data to a new CSV file, for further analysis
gdp_clean_cleaned

In [784]:
print(gdp_clean['Date'].dtype)
print(boe_rate_filtered['Date'].dtype)
print(cpi_q['Date'].dtype)

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


In [793]:
final_df = (
    gdp_clean
    .merge(boe_rate_filtered, on='Date', how='left')
    .merge(cpi_q, on='Date', how='left')
)

In [797]:
print(final_df.head())
print(final_df.info())

     Date  GDP_Level  Bank_Rate  CPI_Inflation
0  2019Q1   665456.0   0.750000          107.8
1  2019Q2   666875.0   0.750000          107.8
2  2019Q3   670547.0   0.750000          107.8
3  2019Q4   670587.0   0.750000          107.8
4  2020Q1   652180.0   0.611719          108.7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype        
---  ------         --------------  -----        
 0   Date           26 non-null     period[Q-DEC]
 1   GDP_Level      26 non-null     float64      
 2   Bank_Rate      26 non-null     float64      
 3   CPI_Inflation  26 non-null     float64      
dtypes: float64(3), period[Q-DEC](1)
memory usage: 964.0 bytes
None


In [798]:
print("GDP dates:")
print(gdp_clean['Date'].min(), gdp_clean['Date'].max())

print("Bank Rate dates:")
print(boe_rate_filtered['Date'].min(), boe_rate_filtered['Date'].max())

print("CPI dates:")
print(cpi_q['Date'].min(), cpi_q['Date'].max())

GDP dates:
2019Q1 2025Q2
Bank Rate dates:
2019Q1 2025Q4
CPI dates:
2019Q1 2025Q4


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [799]:
import os
#try:
  # create your folder here
  # os.makedirs(name='')
#except Exception as e:
 # print(e)
